In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd

In [2]:
!wget --no-check-certificate \
    https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P -O /tmp/sentiment.csv \
    -O /tmp/sentiment.csv
df = pd.read_csv('/tmp/sentiment.csv')

--2021-09-10 18:31:10--  https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P
Resolving drive.google.com (drive.google.com)... 108.177.127.101, 108.177.127.138, 108.177.127.100, ...
Connecting to drive.google.com (drive.google.com)|108.177.127.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/sdmcam01an0eejtcs4luqlbof7p66u4j/1631298600000/11118900490791463723/*/13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P [following]
--2021-09-10 18:31:10--  https://doc-08-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/sdmcam01an0eejtcs4luqlbof7p66u4j/1631298600000/11118900490791463723/*/13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P
Resolving doc-08-ak-docs.googleusercontent.com (doc-08-ak-docs.googleusercontent.com)... 108.177.127.132, 2a00:1450:4013:c07::84
Connecting to doc-08-ak-docs.googleusercontent.com (doc-08-ak-docs.googleusercontent.com)|1

In [3]:
X = df['text']
y = df['sentiment']
from sklearn.model_selection import train_test_split
X_train, X_test , y_train, y_test = train_test_split(X, y , test_size = 0.20)

In [4]:
vocab_size = 10000
oov_token = "<OOV>"
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(X_train)

In [5]:
word_index = tokenizer.word_index

In [6]:
X_train_sequences = tokenizer.texts_to_sequences(X_train)

In [27]:
X_test_sequences = tokenizer.texts_to_sequences(X_test)

In [7]:
padding_type='post'
truncation_type='post'
max_length = 100

In [24]:
X_train_padded= tf.keras.preprocessing.sequence.pad_sequences(
    X_train_sequences, padding="post",truncating='post',maxlen=100
)

In [28]:
X_test_padded= tf.keras.preprocessing.sequence.pad_sequences(
    X_test_sequences, padding="post",truncating='post',maxlen=100
)

In [8]:
!wget --no-check-certificate \
    http://nlp.stanford.edu/data/glove.6B.zip \
    -O /tmp/glove.6B.zip

--2021-09-10 18:34:35--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-09-10 18:34:36--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-09-10 18:34:36--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘/tmp/glove.6B.zip’

/t

In [11]:
import zipfile 
with zipfile.ZipFile('/tmp/glove.6B.zip', 'r') as zip_ref:
    zip_ref.extractall('/tmp/glove')

In [12]:
embeddings_index = {}
f = open('/tmp/glove/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [13]:
embedding_matrix = np.zeros((len(word_index) + 1, max_length))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [14]:
embedding_layer = Embedding(len(word_index) + 1,
                            max_length,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False)

In [15]:
model = Sequential([
    embedding_layer,
    Bidirectional(LSTM(150, return_sequences=True)), 
    Bidirectional(LSTM(150)),
    Dense(6, activation='relu'),
   Dense(1, activation='sigmoid')
])

In [16]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [17]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

In [30]:
%load_ext tensorboard
#rm -rf logs
log_folder = 'logs'
callbacks = [
            EarlyStopping(patience = 10),
            TensorBoard(log_dir=log_folder)
            ]
num_epochs = 10
history = model.fit(X_train_padded, y_train, epochs=num_epochs, validation_data=(X_test_padded, y_test),callbacks=callbacks)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Epoch 1/10
50/50 [==============================] - 29s 573ms/step - loss: 0.6925 - accuracy: 0.4997 - val_loss: 0.6894 - val_accuracy: 0.4762
Epoch 2/10
50/50 [==============================] - 29s 588ms/step - loss: 0.6543 - accuracy: 0.6378 - val_loss: 0.6351 - val_accuracy: 0.7318
Epoch 3/10
50/50 [==============================] - 30s 600ms/step - loss: 0.5981 - accuracy: 0.7320 - val_loss: 0.6057 - val_accuracy: 0.7744
Epoch 4/10
50/50 [==============================] - 30s 596ms/step - loss: 0.5718 - accuracy: 0.7514 - val_loss: 0.5907 - val_accuracy: 0.7644
Epoch 5/10
50/50 [==============================] - 30s 592ms/step - loss: 0.4714 - accuracy: 0.7928 - val_loss: 0.4642 - val_accuracy: 0.7895
Epoch 6/10
50/50 [==============================] - 29s 586ms/step - loss: 0.3996 - accuracy: 0.8280 - val_loss: 0.4458 - val_accuracy: 0.8020
Epoch 7/10
50/50 [==============================] - 

In [31]:
loss, accuracy = model.evaluate(X_test_padded,y_test)
print('Test accuracy :', accuracy)

13/13 [==============================] - 2s 123ms/step - loss: 0.4792 - accuracy: 0.7895
Test accuracy : 0.7894737124443054
